i/o image

In [ ]:
inputs = 'inputs/clip_1.tif'
results = 'results/clip_1.tif'

## Real-ESRGAN

In [ ]:
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import os

In [ ]:
import torch
from models import RealESRGAN
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Read weights for increasing 4 times

In [ ]:
model = RealESRGAN(device, scale=4)
model.load_weights('weights/RealESRGAN_x4.pth', download=True)

converting RGBA to RGB

In [ ]:
image = Image.open(inputs).convert('RGB')

Prediction process

In [ ]:
sr_image = model.predict(image)
sr_image.save(results)

## Geo-information

### Reading georeference information

In [ ]:
import rasterio as rio
from rasterio.transform import from_gcps
from rasterio.control import GroundControlPoint

In [ ]:
da = rio.open(inputs)
dacrs = da.crs
ds = da.read()
print(dacrs)
ds.shape

In [ ]:
x1 = da.xy(0, 0)
x2 = da.xy(0, ds.shape[2])
x3 = da.xy(ds.shape[1],ds.shape[2])
x4 = da.xy(ds.shape[1],0)
x10 = x1[0]
x11 = x1[1]
x20 = x2[0]
x21 = x2[1]
x30 = x3[0]
x31 = x3[1]
x40 = x4[0]
x41 = x4[1]
ds1 = ds.shape[1]*4
ds2 = ds.shape[2]*4

### Georeferencing part

Define ground control point
{ tl : top left; bl : bottom left; br : bottom right; tr : top right }

In [ ]:
tl = GroundControlPoint(0, 0, x10, x11)
bl = GroundControlPoint(ds1, 0, x40, x41)
br = GroundControlPoint(ds1, ds2, x30, x31)
tr = GroundControlPoint(0, ds2, x20, x21)
gcps = [tl, bl, br, tr]

Define CRS

In [ ]:
transform = from_gcps(gcps)
crs = rio.crs.CRS.from_epsg(4326)
crs

Georeferencing

In [ ]:
with rio.open(results, 'r+') as ds:
    ds.crs = crs
    ds.transform = transform

ds = None